
<a href="https://colab.research.google.com/github/UCEMA-QUANt/Data-Science-for-Finance/blob/master/04_preprocesamiento_sklearn1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scikit Learn

Desde hace años, Scikit Learn es el estándar de facto en términos de paquetes de utilidades y algoritmos de aprendizaje automático. Si bien en los últimos tiempos, tanto en gradient boosting como en deep learning, han surgido otros paquetes que son más populares en determinados contextos, esta librería sigue teniendo toda la funcionalidad de soporte para desarrollar y automatizar procesos de entrenamiento y predicción.

Iremos viendo este paquete de una manera funcional, a través de las tareas propias de un proceso de desarrollo e implementación de aprendizaje automático.

#### *Nota sobre la API de sklearn*: la gran mayoría de algoritmos y utilidades son clases que se instancia llamándola como una función con parámetro de inicialización y luego tiene dos métodos, `fit` para entrenar el algoritmo y `transform` o `predict` para aplicarlo, según el caso y la utilidad. Existen algunas funciones simples también para funcionalidades más sencillas. 

## 1. Pre-procesado

Luego de la lectura de datos que ya vimos con `Pandas`, la primera tarea de manipulación de datos suele ser el preprocesador, que consiste en la transformación de los datos para que sean procesables y/o más compatibles con el algoritmo elegido o incluso que le permitan más poder explicativo.

Las transformaciones para lograr que -algunos algoritmos- puedan procesar los datos son la `asignación de nulos` y, en menor medida, la `codificación de variables no intervalares (categóricas y ordinales) `; mientras que el `escalado y normalización` tiene a lograr una mejor compatibilidad con ciertos modelos. Por otro lado, la creación de variables, aunque contiene una gran parte de arte, es una forma muy potente de agregar poder explicativo al modelo.

Veremos estas técnicas en el orden lógico de su utilización, aunque algunas, particularmente las que operan sobre columnas independientes como el escalado y la codificación, son intercambiables.

Asimismo, existen algunas técnicas como la discretización (bins) o la transformación binaria que ya no se usan a menudo; al igual que técnicas de detección de valores extremos (`outliers`) y reducción de dimensionalidad que no tienen a tener ya los mismos beneficios que en otras épocas con menos capacidad de cálculo y algoritmos menos sofisticados. 

### A.- Asignación de nulos o perdidos

Excepto alguna implementación de `gradient boosting`, todos los algoritmos de aprendizaje automático son incompatibles con la existencia de nulos en el dataset. 

Con los valores perdidos hay que ser especialmente cuidadoso, porque a veces tienen de por si un sentido, es decir que no son al azar. Si es así, hay que buscar una forma lógica de completarlos. Si no, se pueden completar de alguna formas distintas.



In [1]:
import numpy as np
import pandas as pd
X = pd.DataFrame([
    [1,2,3,np.nan],
    [np.nan, np.nan, np.nan, 0],
    [-5, 0, 25, np.nan],
    [1,-1, np.nan, np.nan]
], columns=[f"c{i}" for i in range(4)])
X

c0   c1    c2   c3
0  1.0  2.0   3.0  NaN
1  NaN  NaN   NaN  0.0
2 -5.0  0.0  25.0  NaN
3  1.0 -1.0   NaN  NaN

In [2]:
# forma 1 eliminar filas o columnas con muchos nulos
X.dropna(axis=0, thresh=3)

c0   c1    c2  c3
0  1.0  2.0   3.0 NaN
2 -5.0  0.0  25.0 NaN

In [3]:
X.dropna(axis=1, thresh=int(X.shape[0] / 2))

c0   c1    c2
0  1.0  2.0   3.0
1  NaN  NaN   NaN
2 -5.0  0.0  25.0
3  1.0 -1.0   NaN

In [4]:
#completar con algun valro testigo fuera del rango de valores.abs
X.fillna(999)

c0     c1     c2     c3
0    1.0    2.0    3.0  999.0
1  999.0  999.0  999.0    0.0
2   -5.0    0.0   25.0  999.0
3    1.0   -1.0  999.0  999.0

In [5]:
X["c2null"] = X.c2.isnull()
X

c0   c1    c2   c3  c2null
0  1.0  2.0   3.0  NaN   False
1  NaN  NaN   NaN  0.0    True
2 -5.0  0.0  25.0  NaN   False
3  1.0 -1.0   NaN  NaN    True

In [6]:
from sklearn.impute import SimpleImputer

#media
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(X)
res = imputer.transform(X)
res

array([[ 1.        ,  2.        ,  3.        ,  0.        ,  0.        ],
       [-1.        ,  0.33333333, 14.        ,  0.        ,  1.        ],
       [-5.        ,  0.        , 25.        ,  0.        ,  0.        ],
       [ 1.        , -1.        , 14.        ,  0.        ,  1.        ]])

In [7]:
#moda
SimpleImputer(strategy='most_frequent').fit_transform(X)

array([[1.0, 2.0, 3.0, 0.0, False],
       [1.0, -1.0, 3.0, 0.0, True],
       [-5.0, 0.0, 25.0, 0.0, False],
       [1.0, -1.0, 3.0, 0.0, True]], dtype=object)

In [8]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

#con un modelo
imputer = IterativeImputer(max_iter=10, random_state=0)
imputer.fit(X)
imputer.transform(X)

array([[ 1.        ,  2.        ,  3.        ,  0.        ,  0.        ],
       [-0.30826338,  0.33527936,  9.1484619 ,  0.        ,  1.        ],
       [-5.        ,  0.        , 25.        ,  0.        ,  0.        ],
       [ 1.        , -1.        ,  6.30000006,  0.        ,  1.        ]])

In [12]:
X

c0   c1    c2   c3  c2null
0  1.0  2.0   3.0  NaN   False
1  NaN  NaN   NaN  0.0    True
2 -5.0  0.0  25.0  NaN   False
3  1.0 -1.0   NaN  NaN    True

In [11]:
X.interpolate()

c0   c1    c2   c3  c2null
0  1.0  2.0   3.0  NaN   False
1 -2.0  1.0  14.0  0.0    True
2 -5.0  0.0  25.0  0.0   False
3  1.0 -1.0  25.0  0.0    True

In [9]:
from sklearn.impute import MissingIndicator

MissingIndicator(missing_values=999).fit_transform(X.fillna(999))

array([[False, False, False,  True],
       [ True,  True,  True, False],
       [False, False, False,  True],
       [False, False,  True,  True]])

### B.- Codificación de variables ordinales y categorías

Los algoritmos no pueden procesar datos que formato texto o `string`, hay que transformarlos a números. Existen algunas opciones para codificar estas variables, dependiendo de su naturaleza. 

Si son ordinales, fácilmente se pueden traducir a número, en una escala arbitraria y solo manteniendo los ordenes y valores equidistantes. 

Si son categóricas, no es posible hacer lo mismo. Algunas implementaciones de gradient boosting toleran variables categorías, pero en general los demás no. Así que hay que omitirlas o transformarlas.



In [ ]:
X = pd.DataFrame([
    ['M', 'CABA', 'medio'],
    ['M', 'CABA', 'alto'],
    ['F', 'PBA', 'alto'],
    ['F', 'Córdoba', 'bajo'],
    ['F', 'Córdoba', "medio"]
], columns=["sexo", "provincia", "ingreso"])
X

In [ ]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

OrdinalEncoder().fit_transform(X[["sexo", "ingreso"]])

In [ ]:
OrdinalEncoder().fit_transform(X[["sexo", "ingreso"]].sort_values("sexo"))

In [ ]:
encoder = OrdinalEncoder(categories=[["F", "M"], ["bajo", "medio", "alto"]])
encoder.fit_transform(X[["sexo", "ingreso"]])

In [ ]:
X

In [ ]:
encoder.categories

In [ ]:
encoder.inverse_transform([[1, 1]])

In [ ]:
pd.factorize(X.sexo)

In [ ]:
# codificación dummy o one-hot
OneHotEncoder().fit_transform(X[["sexo", "ingreso"]])

In [ ]:
OneHotEncoder().fit_transform(X).toarray()

In [ ]:
X

In [ ]:
encoder = OneHotEncoder().fit(X)
pd.DataFrame(encoder.transform(X).toarray(), columns=[c for cc in encoder.categories_ for c in cc])

In [ ]:
pd.DataFrame(encoder.transform(X).toarray(),
            columns=[
                f"{column}_{category}" for categories, column in zip(encoder.categories_, X.columns)
                for category in categories
            ])

### C.- Escalado y normalización

En casi todos los algoritmos basados en algebra (regresiones y redes neuronales), los parámetros del modelo se encuentran con alguna variante de `gradient descent` y por tanto adolecen se sensibilidad a la escala de los parámetros. Por ello, tiende a ser útil llevar a todas las variables a un rango parejo. Esto normalmente se logra con el escalado de cada columna para que estén en el mismo rango o tengan media y desvíos igual.



In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, Normalizer


X = pd.DataFrame([
    [1,-200,30000],
    [2, 100, 50000],
    [5,0, 100000]
], columns=[f"c{i}" for i in range(3)])
X

In [ ]:
StandardScaler().fit_transform(X)

In [ ]:
StandardScaler().fit_transform(X).mean(axis=0)

In [ ]:
StandardScaler().fit_transform(X).std(axis=0)

In [ ]:
MinMaxScaler().fit_transform(X)

In [ ]:
MaxAbsScaler().fit_transform(X)

In [ ]:
# la normalización es similar pero actua a nivel fila, y se usa cuadno vamos a usar algoritmos que miden distancias entre vectores

Normalizer().fit_transform(X)

In [ ]:
X

In [ ]:
Normalizer().fit_transform(StandardScaler().fit_transform(X))

In [ ]:
transformada = Normalizer().fit_transform(StandardScaler().fit_transform(X))
np.square(transformada)

In [ ]:
np.square(transormada).sum(axis=1)

### Creación de Variables (Feature Engineering)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

X = pd.DataFrame([
    [1,-20],
    [2, 10],
    [5,0]
], columns=[f"c{i}" for i in range(2)])

pd.DataFrame(PolynomialFeatures(2).fit_transform(X),
             columns=["1", "c1", "c2", "c1^2", "c1*c2", "c2^2"])

### Tarea: leer los datos del titanic y transformarlos para lograr el mejor valor de la siguiente funcion

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import train_test_split

def metric(X):
    X_train, X_test, y_train, y_test = train_test_split(X.drop("Survived", axis=1), X["Survived"],
                                                        test_size=0.25, random_state=42)
    model = LogisticRegressionCV()
    model.fit(X_train, y_train)
    return log_loss(y_test,  model.predict_proba(X_test)), accuracy_score(y_test,  model.predict(X_test))


In [ ]:
try:
    data = pd.read_csv("../data/titanic.csv", index_col="PassengerId")
except:
    data = pd.read_csv("https://raw.githubusercontent.com/UCEMA-QUANt/Data-Science-for-Finance/master/data/titanic.csv", index_col="PassengerId")
metric(data.fillna(99999).drop(data.select_dtypes("O"), axis=1))

In [ ]:
accuracy_score(data["Survived"],  np.zeros(data.shape[0]))

In [ ]:
data.isnull().sum()

In [ ]:
data.select_dtypes("O")

In [ ]:
data.Cabin.fillna("NA").value_counts()

In [ ]:
data.Cabin.str[0].fillna("NA").value_counts()

In [ ]:
data.Name.str.split(",")

In [ ]:
data.Name.str.split(",").str[0].value_counts()

In [ ]:
data.Name.value_counts().value_counts()